## Responses
- Agentic API primitive


In [5]:
import os
OPEN_AI_KEY = os.environ.get("OPEN_AI_KEY")
OPEN_AI_MODEL = "gpt-4o-mini"

In [2]:
from openai import  OpenAI
import requests, json


client = OpenAI(api_key=OPEN_AI_KEY)

In [3]:
response = client.responses.create(
    model=OPEN_AI_MODEL,
    input=[{
        "role": "user",
        "content": "Write a one-sentence bedtime story about a unicorn."
    }]
)
print(response)

Response(id='resp_68185bf38ed4819185b9e137596c009a04442f47ac3cfe13', created_at=1746426867.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_68185bf3ad088191974f07101f8053bf04442f47ac3cfe13', content=[ResponseOutputText(annotations=[], text='As the moonlight shimmered over the enchanted meadow, a brave unicorn named Luna soared across the sky, spreading dreams of magic and joy to all the sleeping creatures below.', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=18, input_tokens_deta

In [4]:
response.dict()

/tmp/ipykernel_12298/3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response.dict()


{'id': 'resp_68185bf38ed4819185b9e137596c009a04442f47ac3cfe13',
 'created_at': 1746426867.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'id': 'msg_68185bf3ad088191974f07101f8053bf04442f47ac3cfe13',
   'content': [{'annotations': [],
     'text': 'As the moonlight shimmered over the enchanted meadow, a brave unicorn named Luna soared across the sky, spreading dreams of magic and joy to all the sleeping creatures below.',
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'max_output_tokens': None,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None, 'summary': None},
 'service_tier': 'default',
 'status': 'completed',
 'text': {'format': {'type': 'text'}},
 'truncation': 'disabled',
 '

### Tool call

In [3]:
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(data)
    return data['current']['temperature_2m']

# print(get_weather(latitude="9.9312", longitude="76.2673"))

In [4]:

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celcius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

user_query = [{"role": "user", "content": "What is the weather like in Kochi today?"}]
weather_response = client.responses.create(
    model=OPEN_AI_MODEL,
    input=user_query,
    tools=tools,
)

In [5]:
weather_response.output

[ResponseFunctionToolCall(arguments='{"latitude":9.9312,"longitude":76.2673}', call_id='call_xMz1qxuUDNRZZNUGS8XJZs8k', name='get_weather', type='function_call', id='fc_681856e05d6c81918baca69e2760f71808b47c6ed1e7f7f3', status='completed')]

In [11]:
tool_call = weather_response.output[0]
args = json.loads(tool_call.arguments)

In [12]:
result = globals()[tool_call.name](**args)
result


{'latitude': 9.875, 'longitude': 76.25, 'generationtime_ms': 0.05125999450683594, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 6.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'wind_speed_10m': 'km/h'}, 'current': {'time': '2025-05-05T06:15', 'interval': 900, 'temperature_2m': 32.1, 'wind_speed_10m': 11.7}, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'wind_speed_10m': 'km/h'}, 'hourly': {'time': ['2025-05-05T00:00', '2025-05-05T01:00', '2025-05-05T02:00', '2025-05-05T03:00', '2025-05-05T04:00', '2025-05-05T05:00', '2025-05-05T06:00', '2025-05-05T07:00', '2025-05-05T08:00', '2025-05-05T09:00', '2025-05-05T10:00', '2025-05-05T11:00', '2025-05-05T12:00', '2025-05-05T13:00', '2025-05-05T14:00', '2025-05-05T15:00', '2025-05-05T16:00', '2025-05-05T17:00', '2025-05-05T18:00', '2025-05-05T19:00', '2025-05-05T20:00', '2025-05-05T21:00', '2025-05-05T22:00', '2025-05-05

32.1